**Libraries**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

import conv_deconv_blocks as cdb
import data_processing as dp
import gan_cvae_model as md
import cris_utils as cu

**Tensorflow GPU Verification**

In [ ]:
print("\n","Versión de tensorflow: " + str(tf.__version__),"\n")
print("Comprobación de la GPU:",tf.config.experimental.list_physical_devices('GPU'),"\n")

# GAN CVAE

**Model Arquitecture**

In [ ]:
N_H = 64
N_W = 64
N_C = 1

cvae_enc_convs = [(16,True,cdb.lr_act,3,2),
              (32,False,cdb.lr_act,3,2),
              (64,True,cdb.lr_act,3,2),
             (128,False,cdb.lr_act,3,2),
             (256,False,cdb.lr_act,3,2)]

cvae_dec_deconvs = [(256,True,cdb.r_act,3,2),
               (128,False,cdb.r_act,3,2),
               (64,True,cdb.r_act,3,2),
               (32,False,cdb.r_act,3,2),
               (N_C,False,cdb.th_act,3,2)]

gan_disc_convs = [(16,True,cdb.lr_act,3,2),
              (32,False,cdb.lr_act,3,2),
              (64,True,cdb.lr_act,3,2),
             (128,False,cdb.lr_act,3,2),
             (256,False,cdb.lr_act,3,2)]

n_cvae_conv_ls = len(cvae_enc_convs)
n_cvae_deconv_ls = len(cvae_dec_deconvs)

max_layers = cu.max_conv_deconv_layers(N_H)
assert (n_cvae_conv_ls<=max_layers and n_cvae_deconv_ls<=max_layers), "Incorrect number of layers for cvae model"

config = {
    md.batch_size_k: 16,
    md.latent_dim_k: 32,
    md.data_dir_patt_k: ["data_FVC2006","png"],
    md.fps_shape_k: (N_H,N_W,N_C),
    md.cvae_checkpoints_folder_k: "./tf_Checkpoints/CVAE",
    md.gan_checkpoints_folder_k: "./tf_Checkpoints/GAN",
    md.max_checkpoints_k: 2
}

cvae_enc_config = {
    cdb.fps_shape_k: config[md.fps_shape_k],
    cdb.enc_dec_lys_info_k: cvae_enc_convs,
    cdb.enc_fin_den_len_k: 2*config[md.latent_dim_k]
}

hw_f_v = N_H/np.power(2,n_cvae_deconv_ls)

cvae_dec_config = {
    cdb.dec_den_info_k: ([hw_f_v,hw_f_v,256],False,cdb.r_act,(config[md.latent_dim_k],)),
    cdb.enc_dec_lys_info_k: cvae_dec_deconvs,
}

gan_disc_config = {
    cdb.fps_shape_k: config[md.fps_shape_k],
    cdb.enc_dec_lys_info_k: gan_disc_convs,
    cdb.enc_fin_den_len_k: 1
}

**Data Preparation**

In [ ]:
ds_data_dirs = dp.load_process_fp_dataset(config[md.data_dir_patt_k],config[md.fps_shape_k],config[md.batch_size_k])
l = list(ds_data_dirs.as_numpy_iterator())
cu.imshow(l[0][0,:])

**Creation of Model**

In [ ]:
gan_cvae = md.GAN_CVAE(cvae_enc_config,cvae_dec_config,gan_disc_config,config)

**CVAE Training Configuration**

In [ ]:
cvae_train_conf = {
    md.num_epochs_k: 5,
    md.num_images_k: 5,
    md.checkpoints_frecuency_k: 5,
    md.use_latest_checkpoint_k: False,
    md.types_losses_k: [md.square_loss,md.kl_loss,md.ssim_loss],
    md.alphas_losses_k: [1.0,0.01,0.5],
    md.dataset_k: ds_data_dirs,
    md.tensorboard_folder_k: "./tf_Tensorboard_logs/CVAE",
    md.out_images_folder_k: "./img_Performance_Images/CVAE",
}

**CVAE Training**

In [ ]:
#gan_cvae.cvae_train(cvae_train_conf)

**GAN Training Configuration**

In [ ]:
gan_train_conf = {
    md.num_epochs_k: 5,
    md.num_images_k: 5,
    md.checkpoints_frecuency_k: 5,
    md.use_latest_checkpoint_k: False,
    md.dataset_k: ds_data_dirs,
    md.tensorboard_folder_k: "./tf_Tensorboard_logs/GAN",
    md.out_images_folder_k: "./img_Performance_Images/GAN",
}

**Gan Training**

In [ ]:
gan_cvae.gan_train(gan_train_conf)

**Remove Files**

In [1]:
import cris_utils as cu
code_dir = "C:/Users/Y50/Documents/Universidad/Tesis/Codigo/"
cu.delete_All(code_dir+"tf_Tensorboard_logs/CVAE/*")
cu.delete_All(code_dir+"tf_Tensorboard_logs/GAN/*")
cu.delete_All(code_dir+"tf_Checkpoints/CVAE/*")
cu.delete_All(code_dir+"tf_Checkpoints/GAN/*")
cu.delete_All(code_dir+"img_Performance_Images/CVAE/*")
cu.delete_All(code_dir+"img_Performance_Images/GAN/*")

Are you sure you want to delete this? 

Type y or n: y

 Deleted Files
Are you sure you want to delete this? 

Type y or n: y

 Deleted Files
Are you sure you want to delete this? 

Type y or n: y

 Deleted Files
Are you sure you want to delete this? 

Type y or n: y

 Deleted Files
Are you sure you want to delete this? 

Type y or n: y

 Deleted Files
Are you sure you want to delete this? 

Type y or n: y

 Deleted Files


**Test Code**

In [ ]:
b1 = next(iter(ds_data_dirs))

In [ ]:
ls = gan_cvae.gan_discriminator.layers

for l in ls:
    print(l.name)

print(ls[1].moving_mean)